In [5]:
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from algorithm_utils import *
import general_utils as gu

In [94]:
testing_length = 100
start_index = 4500
pair_df: pd.DataFrame = gu.load_local_data().iloc[start_index:start_index + testing_length].reset_index(drop=True)

In [95]:
zigzag_df = zigzag(pair_df)
h_o_zigzag_df = generate_h_o_zigzag(zigzag_df)
pbos_df = h_o_zigzag_df.loc[h_o_zigzag_df.is_pbos]
bos_df = find_confirmed_boss(pbos_df, pair_df)
lpl_df = find_lpls(bos_df, zigzag_df)
lplb_df = find_lplbs(bos_df, zigzag_df)

boxes_list = []
for lpl in lpl_df.itertuples():
    start_index = lpl.pair_df_index
    base_candle = Candle.create(pair_df.iloc[lpl.pair_df_index])
    box_type = "long" if lpl.pivot_type == "valley" else "short"
    box_to_add = Box(base_candle, box_type)
    box_to_add.check_box_entries(pair_df)
    boxes_list.append(box_to_add)

In [99]:
pt = gu.PlottingTool()
pt.draw_candlesticks(pair_df)
pt.draw_zigzag(zigzag_df)
pt.draw_zigzag(h_o_zigzag_df, title='Higher order Zigzag', color='black')
# pt.draw_points_with_label(pbos_df.loc[pbos_df.pivot_type == "peak"], "PBOS")
pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="red")
pt.draw_points_with_label(lplb_df.loc[lplb_df.pivot_type == "peak"], "LPLB-S", color="orange")
pt.draw_points_with_label(lpl_df.loc[lpl_df.pivot_type == "peak"], "LPL-S", color="green")
for box in boxes_list:
    print(box.price_exit_index)
    pt.draw_box(box, pair_df_end_index=pair_df.last_valid_index())
# pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="blue")
pt.show()

74
